In [6]:
from gad_adversarial_robustness.gad.dominant import dominant
from pygod.utils import load_data
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from sklearn.metrics import roc_auc_score
import os
from gad_adversarial_robustness.utils.graph_utils import prepare_graph
from torch_geometric.data import Data
import numpy as np
from gad_adversarial_robustness.utils.graph_utils import get_n_anomaly_indexes
from typing import List
from gad_adversarial_robustness.poison.greedy import multiple_AS

script_dir = os.path.abspath('')


dataset_caching_path = os.path.join(script_dir, '..', '..', '..', 'data')

## run the poison

In [7]:




import copy
from gad_adversarial_robustness.poison.greedy import poison_attack


clean_data: Data = load_data("inj_cora", dataset_caching_path)
poisoned_data: Data = load_data("inj_cora", dataset_caching_path)

truth: List[int] = poisoned_data.y.bool()
amount_of_nodes = poisoned_data.num_nodes
_, adj, _ = prepare_graph(poisoned_data)
dense_adj = adj.to_dense()  #Fill in zeroes where there are no edges


print("Create poison compatible adjacency matrix...")
triple = []
for i in range(amount_of_nodes):
    for j in range(i + 1, amount_of_nodes):
        triple.append([i, j, dense_adj[i,j]])  #Fill with 0, then insert actual after



triple = np.array(triple)

# These are the nodes we try reduce the active subnetwork score for (disguising anonomaly nodes)
target_node_lst = get_n_anomaly_indexes(truth, 999)
#target_node_lst = get_n_anomaly_indexes(truth, 10)
print(type(target_node_lst)), print(f'target node list: {target_node_lst}'), print(target_node_lst)

print("Making model...")
model = multiple_AS(target_lst = target_node_lst, n_node = amount_of_nodes, device = 'cpu')
budget = 2  # The amount of edges to change


print("Starting attack...")
#triple = triple.detach().cpu().numpy()

#triple = torch.tensor(triple, requires_grad=True)


adj_adversary, _, _ = poison_attack(model, triple, budget)

print("Converting to compatible tensor...")

# Create Edge Index'
edge_index = torch.tensor([[],[]])

# Transpose it to make shape compatible
transposed_adj_adversary = torch.transpose(adj_adversary, 0, 1)

for i in range(len(adj_adversary)):
    if(adj_adversary[i][2] != 0):   #If edge value is not 0 (no edge)
        #Add edge to edge index, choosing first 2 elements (edges), and then the ith edge
        edge_index = torch.cat((edge_index, transposed_adj_adversary[:2, i:i+1]), -1)
        # Dataset uses edges both ways so add reverse edge as well
        edge_index = torch.cat((edge_index, torch.flip(transposed_adj_adversary[:2, i:i+1], dims=[0])), -1)


edge_index = edge_index.type(torch.int64)
poisoned_data.edge_index = edge_index



tensor([1., 1., 1.,  ..., 1., 1., 1.])
node_attrs
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
adj
SparseTensor(row=tensor([   0,    0,    0,  ..., 2707, 2707, 2707]),
             col=tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]),
             val=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
             size=(2708, 2708), nnz=11054, density=0.15%)
labels
tensor([0, 0, 0,  ..., 0, 0, 0])
Create poison compatible adjacency matrix...
Anomalies indexes: [  10   50   70   76  104  124  127  143  151  170  179  181  196  214
  217  224  227  287  289  294  311  333  425  438  451  454  459  539
  565  572  578  581  615  619  641  652  654  660  670  674  692  711
  722  738  781  833  843  869  874  878  882  891  895  915  923  938
  980  982  996 1002 1014 1035 1053 1079 1090 1096 1133 1135 1206 1211
 1224 1229 1235 1287 1293 1310 1362 1391 1414 1426 1533 1540 1543 1547
 1570 1

c:\users\andwh\documents\p6\aau-dat6-poison-gad\gad_adversarial_robustness\poison\greedy.py:35: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  return torch.sparse.mm(torch.sparse.mm(A_sp, A_sp), A_sp).to_dense()
c:\users\andwh\documents\p6\aau-dat6-poison-gad\gad_adversarial_robustness\poison\greedy.py:39: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3618.)
  E = torch.sum(A, 1) + 0.5 * torch.diag(self.spars

Converting to compatible tensor...


In [9]:
import yaml
from gad_adversarial_robustness.gad.dominant.dominant import Dominant
from gad_adversarial_robustness.utils.graph_utils import load_anomaly_detection_dataset

yaml_path = os.path.join(script_dir, '..', 'configs', 'dominant_config.yaml')
with open(yaml_path) as file:
        config = yaml.safe_load(file)

"""
Dominant on clean data
"""

adj, attrs, label, adj_label = load_anomaly_detection_dataset(clean_data)
adj = torch.FloatTensor(adj)
adj_label = torch.FloatTensor(adj_label)
attrs = torch.FloatTensor(attrs)


model = Dominant(feat_size=attrs.size(1), hidden_size=config['hidden_dim'], dropout=config['dropout'], 
                 device=config['device'], adj=adj, adj_label=adj_label, attrs=attrs, label=label)
model.fit(config)

"""
Dominant on poisoned data
"""

adj, attrs, label, adj_label = load_anomaly_detection_dataset(poisoned_data)
adj = torch.FloatTensor(adj)
adj_label = torch.FloatTensor(adj_label)
attrs = torch.FloatTensor(attrs)


model = Dominant(feat_size=attrs.size(1), hidden_size=config['hidden_dim'], dropout=config['dropout'], 
                 device=config['device'], adj=adj, adj_label=adj_label, attrs=attrs, label=label)
model.fit(config)



Epoch: 0000, train_loss=2.17160, train/struct_loss=8.73512, train/feat_loss=0.53073
Epoch: 0000, Auc: 0.8461822590650201
Epoch: 0001, train_loss=1.32507, train/struct_loss=5.39569, train/feat_loss=0.30742
Epoch: 0002, train_loss=0.91094, train/struct_loss=3.51874, train/feat_loss=0.25899
Epoch: 0003, train_loss=0.71144, train/struct_loss=2.52575, train/feat_loss=0.25786
Epoch: 0004, train_loss=0.64340, train/struct_loss=2.18490, train/feat_loss=0.25802
Epoch: 0005, train_loss=0.63391, train/struct_loss=2.13861, train/feat_loss=0.25774
Epoch: 0006, train_loss=0.63444, train/struct_loss=2.14121, train/feat_loss=0.25774
Epoch: 0007, train_loss=0.63428, train/struct_loss=2.14026, train/feat_loss=0.25778
Epoch: 0008, train_loss=0.63376, train/struct_loss=2.13779, train/feat_loss=0.25776
Epoch: 0009, train_loss=0.63376, train/struct_loss=2.13790, train/feat_loss=0.25773
Epoch: 0010, train_loss=0.63387, train/struct_loss=2.13870, train/feat_loss=0.25766
Epoch: 0010, Auc: 0.8389358822534257
Ep